<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/%EA%B3%A0%EA%B0%9D_%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89_%EB%B6%84%EB%A5%98_AI_%ED%95%B4%EC%BB%A4%ED%86%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Info.

train.csv [파일]
고객 관련 금융 정보
ID : 대출 고객의 고유 ID
대출등급 : 예측 목표


test.csv [파일]
고객 관련 금융 정보
ID : 대출 고객의 고유 ID
대출등급이 존재하지 않음


sample_submission.csv [파일] - 제출 양식
ID : 대출 고객의 고유 ID
대출등급 : test.csv에서 제공된 고객의 대출등급을 예측하여 기입

1. 파생변수 생성
- EDA를 통해 '총상환원금', '총상환이자'가 주요한 영향을 미치는 것을 파악했고, 두 변수의 결합('총상환원금'/'총상환이자')으로 파생변수를 생성했습니다.
- 신용점수 평가 기준을 찾아보니 상환능력이 중요한 것 같더군요. 따라서, '총상환원금'/'대출금액'으로 '상환비율' 변수를 생성했습니다.
- 두 변수 생성 이후, 0.83에서 0.951로 점수가 가장 큰 폭으로 상승했습니다.

2.  변수 제거 (RFECV, Feature Importance)
- Feature Importance를 찍어보니 값이 0에 가까운 변수들이 몇 개 있어서 아예 제거하기로 했습니다.
- 변수 선택법인 RFECV를 활용, 변수를 3개로 줄였습니다. ('대출기간', '총상환원금/총상환이자', '상환비율')
- 이 방법으로 0.02 정도 점수가 오른 것 같습니다.

3. 하이퍼파라미터 튜닝 (Optuna)
- 튜닝 전, 성능이 괜찮게 나왔던 ET, RF, DT, XGB 총 4가지 모델들에 대해 튜닝을 진행했습니다.
- XGB의 경우 tree_method에 따라 점수가 많이 달라지는 것을 확인해 튜닝 작업에 추가했습니다.
- 튜닝 이후, 0.951에서 0.953으로 점수가 상승했습니다.

4. 앙상블 모델 (Stacking)
- Voting과 Stacking, 그리고 여러 모델끼리 조합을 해보면서 가장 성능이 좋은 조합을 찾아갔습니다.
- Stacking(ET+XGB+DT+RF)을 사용한 결과, 0.953에서 0.955로 점수가 증가해 최종 모델로 선정했습니다.v

In [ ]:
import pandas as pd
import numpy as np
import random
import os
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything()

from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.feature_selection import RFECV

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_raw = pd.read_csv('train.csv')
test_raw = pd.read_csv('test.csv')

In [ ]:
def preprocess(train, test):

    ############################################### train 전처리 ###############################################
    train = train.drop(columns = ['ID'])
    train['대출기간'] = train['대출기간'].str.replace('[^0-9]','')
    train['대출기간'] = (train['대출기간'].astype(int))//12

    # 범주형 변수 인코딩
    le = LabelEncoder()
    train['대출등급'] = le.fit_transform(train['대출등급'])

    # 파생변수
    train['총상환원금/총상환이자'] = (train['총상환원금'])/(train['총상환이자']+1)
    train['상환비율'] = train['총상환원금'] / train['대출금액']


    ############################################### test 전처리 ###############################################
    test = test.drop(columns = ['ID'])
    test['대출기간'] = test['대출기간'].str.replace('[^0-9]','')
    test['대출기간'] = (test['대출기간'].astype(int))//12

    # 파생변수
    test['총상환원금/총상환이자'] = (test['총상환원금'])/(test['총상환이자']+1)
    test['상환비율'] = test['총상환원금'] / test['대출금액']

    return train, test

In [ ]:
train,test = preprocess(train_raw, test_raw)

In [ ]:
X = train[['대출기간','총상환원금/총상환이자','상환비율']]; y = train['대출등급']
test = test[['대출기간','총상환원금/총상환이자','상환비율']]

In [ ]:
rf = RandomForestClassifier(random_state = 42
                         , n_estimators = 305
                         , criterion = 'gini'
                         , max_depth = 62
                         , min_samples_split = 7
                         , min_samples_leaf = 1)
dt = DecisionTreeClassifier(random_state = 42
                         , criterion = 'entropy'
                         , max_depth = 25
                         , min_samples_split = 2
                         , min_samples_leaf = 1)
et = ExtraTreesClassifier(random_state = 42
                         , n_estimators = 930
                         , criterion = 'entropy'
                         , max_depth = 65
                         , min_samples_split = 6
                         , min_samples_leaf = 1
                         )
xgb = XGBClassifier(random_state = 42
                   , n_estimators = 665
                   , reg_lambda = 0.04614513317156364
                   , reg_alpha = 0.8831857977740336
                   , tree_method = "exact"
                   , colsample_bytree = 0.7664006730032823
                   , subsample = 0.6579847353498132
                   , learning_rate = 0.4046062291148477
                   , max_depth = 64
                   , min_child_weight = 2
                   )

In [ ]:
scale = StandardScaler()
X = scale.fit_transform(X)
test = scale.transform(test)

In [ ]:
estimators = [('et',et), ('xgb',xgb), ('dt',dt), ('rf',rf)]
stack = StackingClassifier(estimators, final_estimator=LogisticRegression(), verbose=1)
stack.fit(X,y)
pred = stack.predict(test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  5.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.9min finished


In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub['대출등급'] = pred
sub['대출등급'] = sub['대출등급'].map({0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G'})
sub.head()

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,B
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C


In [ ]:
sub.to_csv('final_submission.csv', index = False)

<배울 점>
- Optuna 라이브러리 이용한 점
- 두 변수의 결합('총상환원금'/'총상환이자')으로 파생변수를 생성했다는 점